In [19]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

# Create a temporary directory to store code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Docker image to use.
    timeout=10,  # Timeout for each code execution.
    work_dir=temp_dir.name,  # Temporary directory for storing files.
)

# Create an agent with Docker execution configuration.
code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for safety.
)

# Example code to execute in the container.
code = """
print('Hello from Docker!')
"""

# Execute the code using the agent's `run_code` method.
code_executor_agent_using_docker.run_code(code)


(0,
 'Hello from Docker!\n',
 'python:tmp_code_8de0dc0b8cc94d696bbd19129d125a54.py')

In [13]:
# Clean up when no longer needed.
executor.stop()
temp_dir.cleanup()

In [20]:
import os
# The code writer agent's system message is to instruct the LLM on how to use
# the code executor in the code executor agent.
code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

code_writer_agent = ConversableAgent(
    "code_writer_agent",
    system_message=code_writer_system_message,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": "API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    code_execution_config=False,  # Turn off code execution for this agent.
)

In [26]:
code_executor_agent=ConversableAgent(
    "code_executor_agent",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": "API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    code_execution_config={"executor":executor},
    human_input_mode="Always"
)

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message="Write Python code to calculate the 14th Fibonacci number.",
)

code_executor_agent (to code_writer_agent):

Write Python code to calculate the 14th Fibonacci number.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer_agent (to code_executor_agent):

Here is a Python code block that uses a function to calculate the 14th Fibonacci number.

```python
def fibonacci(n):
    if n <= 0:
        return "Input should be a positive integer"
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b

print(fibonacci(14))
```

This function starts by defining the first two Fibonacci numbers, 0 and 1. It then enters a loop that runs `n-2` times (since we've already defined the first two numbers), in which it continually reassigns `a` and `b` to `b` and `a + b`, respectively, effectively moving two steps down the Fibonacci sequence with each iteration. The result is the `n`th F

: 